<a href="https://colab.research.google.com/github/gabsrcha/Gabrielly_PIBIC_Lung.Node.Segmentation/blob/main/C%C3%B3pia_de_Gabs_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch import randint
!pip install torchmetrics
from torchmetrics.classification import Dice
from sklearn.metrics import accuracy_score
import torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 9.5 MB/s eta 0:00:00


In [2]:
pip freeze > requirements.txt

In [3]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn()
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features , 2)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask , hidden_layer , 2)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 103MB/s]


In [4]:
transform = T.ToTensor()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_images = sorted(os.listdir("/content/drive/MyDrive/Gabs_lung/train/images/"))
train_mask = sorted(os.listdir("/content/drive/MyDrive/Gabs_lung/train/mask/"))
train_imgs = np.array(train_images)
train_masks = np.array(train_mask)


valid_images = sorted(os.listdir("/content/drive/MyDrive/Gabs_lung/valid/images/"))
valid_mask = sorted(os.listdir("/content/drive/MyDrive/Gabs_lung/valid/mask/"))
val_imgs = np.array(valid_images)
val_masks = np.array(valid_mask)


In [7]:
def custom_collate(data):
#   data = [d for d in data if d[1]["boxes"].shape[0] > 0]
  return data

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
model.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu):

In [11]:
params = [p for p in model.parameters() if p.requires_grad]

#criterion = nn.L1Loss.to(device)
#optimizer = torch.optim.SGD(params, lr=0.0001, momentum=0.9, weight_decay=0.005)

optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0.005)

In [12]:
class CustDat(torch.utils.data.Dataset):
    def __init__(self , images , masks):
        self.imgs = images
        self.masks = masks

    def __getitem__(self , idx):
        img = Image.open("/content/drive/MyDrive/Gabs_lung/train/images/" + self.imgs[idx]).convert("RGB")
        mask = Image.open("/content/drive/MyDrive/Gabs_lung/train/mask/" + self.masks[idx])
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)
        masks = np.zeros((num_objs , mask.shape[0] , mask.shape[1]))
        #print(np.where(mask > np.min(mask)))
        for i in range(num_objs):
            #masks[i][mask == i+1] = True
            masks[i][mask > 0] = True
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin , ymin , xmax , ymax])
        boxes = torch.as_tensor(boxes , dtype = torch.float32)
        labels = torch.ones((num_objs,) , dtype = torch.int64)
        masks = torch.as_tensor(masks , dtype = torch.uint8)
        #print(boxes)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        return T.ToTensor()(img) , target

    def __len__(self):
        return len(self.imgs)


class CustDatVal(torch.utils.data.Dataset):
    def __init__(self , images , masks):
        self.imgs = images
        self.masks = masks

    def __getitem__(self , idx):
        img = Image.open("/content/drive/MyDrive/Gabs_lung/valid/images/" + self.imgs[idx]).convert("RGB")
        mask = Image.open("/content/drive/MyDrive/Gabs_lung/valid/mask/" + self.masks[idx])
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)
        masks = np.zeros((num_objs , mask.shape[0] , mask.shape[1]))
        for i in range(num_objs):
            #masks[i][mask == i+1] = True
            masks[i][mask > 0] = True
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin , ymin , xmax , ymax])
        boxes = torch.as_tensor(boxes , dtype = torch.float32)
        labels = torch.ones((num_objs,) , dtype = torch.int64)
        masks = torch.as_tensor(masks , dtype = torch.uint8)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        return T.ToTensor()(img) , target

    def __len__(self):
        return len(self.imgs)

In [13]:
train_dl = torch.utils.data.DataLoader(CustDat(train_imgs , train_masks),
                                 batch_size = 100,
                                 shuffle = True ,
                                 collate_fn = custom_collate ,
                                 num_workers = 1 ,
                                 pin_memory = True if torch.cuda.is_available() else False)

val_dl = torch.utils.data.DataLoader(CustDatVal(val_imgs , val_masks) ,
                                 batch_size = 51,
                                 shuffle = True ,
                                 collate_fn = custom_collate ,
                                 num_workers = 1 ,
                                 pin_memory = True if torch.cuda.is_available() else False)


In [14]:
print(enumerate(train_dl))

In [ ]:
#train_dl = torch.utils.data.DataLoader(CustDat(train_imgs , train_masks))
#val_dl = torch.utils.data.DataLoader(CustDatVal(val_imgs , val_masks))

all_train_losses = []
all_val_losses = []
flag = False
for epoch in range(600):
    #print(epoch)
    train_epoch_loss = 0
    val_epoch_loss = 0
    model.train()
    for i , dt in enumerate(train_dl):
        if len(dt) <= 1:
            print("++++",i)
            continue
        imgs = [dt[0][0].to(device) , dt[1][0].to(device)]
        targ = [dt[0][1] , dt[1][1]]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targ]
        loss = model(imgs , targets)
        if not flag:
            print(loss)
            flag = True
        losses = sum([l for l in loss.values()])
        train_epoch_loss += losses.cpu().detach().numpy()
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    all_train_losses.append(train_epoch_loss)
    with torch.no_grad():
        for j , dt in enumerate(val_dl):
            imgs = [dt[0][0].to(device) , dt[1][0].to(device)]
            targ = [dt[0][1] , dt[1][1]]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targ]
            loss = model(imgs , targets)
            losses = sum([l for l in loss.values()])
            val_epoch_loss += losses.cpu().detach().numpy()
        all_val_losses.append(val_epoch_loss)
    print(epoch , "  " , train_epoch_loss , "  " , val_epoch_loss)

{'loss_classifier': tensor(0.7294, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0001, grad_fn=<DivBackward0>), 'loss_mask': tensor(1.2687, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_objectness': tensor(0.6965, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0011, grad_fn=<DivBackward0>)}
0    4.498820900917053    2.1282553672790527
1    4.559898614883423    1.392319679260254
2    2.7249547243118286    1.4161605834960938
3    2.4741194248199463    1.1204700469970703
4    2.0425443053245544    1.526381492614746
5    1.9102196097373962    1.083957552909851
6    2.77347195148468    1.2170971632003784
7    2.0741308331489563    0.8001686930656433
8    1.6201080083847046    0.8185805678367615
9    1.317038893699646    0.995917558670044
10    2.0749118328094482    0.896856427192688
11    1.822017788887024    0.8770217895507812
12    1.605968952178955    0.7595527172088623
13    1.6203755140304565    0.7790910005569458
14    1.40875917673110

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(all_train_losses, label='Train')
plt.plot(all_val_losses, label='Test', alpha=0.5)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Convergence', fontsize=16)
plt.legend()
plt.show()


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Gabs_lung/modelgabs6.pt')

In [ ]:
images = sorted(os.listdir("/content/drive/MyDrive/Gabs_lung/general/images/"))

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Gabs_lung/modelgabs6.pt'))
model.eval()

In [ ]:
# Example plotting multiple values
metric = Dice()
values = [ ]
for _ in range(100):
  values.append(metric(randint(2,(10,)), randint(2,(10,))))
fig_, ax_ = metric.plot(values)

In [ ]:
from torchvision.models.detection import image_list

from torch import nn


#criterion = nn.CrossEntropyLoss.to(device)
l=0

model.eval()

#while (l<61):
#for l in range(1,len(images)):
f = plt.figure()
all_test_losses = []
for l in images:
    print(l)
    img = Image.open("/content/drive/MyDrive/Gabs_lung/general/images/"+str(l))
    mask = Image.open("/content/drive/MyDrive/Gabs_lung/general/mask/"+str(l))
    transform = T.ToTensor()
    ig = transform(img)
    with torch.no_grad():
        pred = model([ig.to(device)])
        try:
          pred1 = (pred[0]["masks"][0].cpu())
          # Converte as máscaras para arrays numpy
          pred_mask = (pred[0]["masks"][0].cpu().numpy() > 0).astype(int)
          true_mask = (np.array(mask) > 0).astype(int)
          loss = nn.CrossEntropyLoss(pred_mask, true_mask)
          all_test_losses.append(loss.cpu.data)

          # Calcula o valor Dice
          dice_metric = torchmetrics.classification.Dice()
          dice_value = dice_metric(torch.from_numpy(pred_mask), torch.from_numpy(true_mask))
          print(f"Valor Dice: {dice_value.item()}")

          # Calcula a acurácia
          accuracy = accuracy_score(true_mask.flatten(), pred_mask.flatten())
          print(f"Acurácia: {accuracy}")
          #plt.imshow((pred[0]["masks"][0].cpu().detach().numpy()*255).astype("uint8").squeeze(), cmap='gray')

          # # Subplot para a máscara
          plt.subplot(1, 3, 1)
          plt.imshow((pred[0]["masks"][0].cpu().detach().numpy() * 255).astype("uint8").squeeze(), cmap='gray')
          plt.axis('off')
          plt.title("Máscara")

          # # Subplot para a imagem original
          plt.subplot(1, 3, 2)
          plt.imshow(mask)
          plt.axis('off')
          plt.title("Máscara Original")

          # # Subplot para a imagem original
          plt.subplot(1, 3, 3)
          plt.imshow(img)
          plt.axis('off')
          plt.title("Imagem Original")

          #plt.savefig("/content/drive/MyDrive/Gabs_lung/test6/"+str(l), format='png', bbox_inches='tight',pad_inches=0)
          plt.clf()
        except IndexError:
            pass
epoch_loss = np.asarray(epoch_loss)

In [ ]:
from sklearn.metrics import accuracy_score
import torchmetrics
import numpy as np

# ...

# Após a geração das máscaras pelo modelo
with torch.no_grad():
    pred = model([ig.to(device)])
    try:
        # Converte as máscaras para arrays numpy
        pred_mask = (pred[0]["masks"][0].cpu().numpy() > 0).astype(int)
        true_mask = (np.array(mask) > 0).astype(int)

        # Calcula o valor Dice
        dice_metric = torchmetrics.classification.Dice()
        dice_value = dice_metric(torch.from_numpy(pred_mask), torch.from_numpy(true_mask))
        print(f"Valor Dice: {dice_value.item()}")

        # Calcula a acurácia
        accuracy = accuracy_score(true_mask.flatten(), pred_mask.flatten())
        print(f"Acurácia: {accuracy}")

        # Continua com a geração das imagens e outros processamentos
    except IndexError:
        pass
